In [4]:
# -*- coding: utf-8 -*-
"""Generate a *DefMap* video (heatâ€‘map of displacement speed) with the
**Lucasâ€‘Kanade** opticalâ€‘flow algorithm â€“Â in exakt dem Stil deines
Referenzâ€‘PNG (plasma colourâ€‘map, Colorâ€‘Bar rechts, Titel oben).

Workflow
========
1. Lade das Timelapseâ€‘Video.
2. Croppe einen konfigurierbaren ROI, damit nur der relevante Bildausschnitt
   ausgewertet wird.
3. Berechne **dichten** Lucasâ€‘Kanadeâ€‘Fluss: wir legen ein gleichmÃ¤ssiges
   Gitternetz von Seedâ€‘Punkten (Rasterweite *GRID_STEP*) und propagieren es
   frameâ€‘fÃ¼râ€‘frame mit `cv2.calcOpticalFlowPyrLK`.  Die resultierenden sparse
   Vektoren werden in zwei 2â€‘Dâ€‘Matrizen (dx,â€¯dy) projiziert und anschliessend
   per GauÃŸâ€‘Blur geglÃ¤ttet, um ein pseudodichtes Feld zu erhalten.
4. Leite daraus die Geschwindigkeit â€–vâ€– ab.
5. Rendere jedes Frame mit Matplotlib (plasmaâ€‘Colormap; fester Wertebereich
   0Â â†’Â blau bis *MAX_SPEED_VIS*Â â†’Â gelb) â€“Â Titel: *DefMap at Time:Â t*; Colorâ€‘Bar
   rechts.
6. Schreibe das Ganze als MP4 *und* speichere Basisâ€‘Metriken pro Frame in
   einer CSV.

Dieser Ansatz ist etwas langsamer als FarnebÃ¤ck, produziert aber das gewÃ¼nschte
Lucasâ€‘Kanadeâ€‘basiertes Ergebnis.  FÃ¼r Sequenzen mit >1000Â Frames ggf.
*DOWNSAMPLE* erhÃ¶hen.
"""
from __future__ import annotations

import cv2
import numpy as np
import pandas as pd
import matplotlib
matplotlib.use("Agg")  # headâ€‘less backend (kein GUI nÃ¶tig)
import matplotlib.pyplot as plt
from pathlib import Path

# ----------------------------------------------------------------------------
# CONFIGURATION
# ----------------------------------------------------------------------------
ROOT_DIR = Path("/Users/macbook/Library/Mobile Documents/com~apple~CloudDocs/Studium/Spezialisierung/CodeExampleRayan")
INPUT_VIDEO = ROOT_DIR / "data/input/LinearStackAlignmentSift_Gauss5px.avi"
OUTPUT_DIR = ROOT_DIR / "data/output/defmap_video_lk"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
VIDEO_OUT = OUTPUT_DIR / "defmap_lk_plasma.mp4"
CSV_OUT = OUTPUT_DIR / "defmap_lk_speed_metrics.csv"

ROI_MARGIN = 100      # RandÂ­abstand in Pixeln, der weggeschnitten wird
GRID_STEP = 5         # Gitterabstand (Pixel) der Seedâ€‘Punkte
SMOOTH_K = 5          # KernelgrÃ¶ÃŸe fÃ¼r GauÃŸâ€‘Blur â†’ glÃ¤ttet das sparse Feld
DOWNSAMPLE = 1        # jede nâ€‘te Frame verarbeiten (>=1)
FPS_OUT = 10          # FPS des Ausgabevideos
MAX_SPEED_VIS = 10.0  # oberes Ende der Colorâ€‘Bar (Âµm oder Pixel pro Frame)
FIG_DPI = 150         # AuflÃ¶sung der Matplotlibâ€‘Figures

# Lucasâ€‘Kanadeâ€‘Parameter (bewÃ¤hrt fÃ¼r Zellâ€‘Timelapse)
LK_ARGS = dict(
    winSize=(11, 11),
    maxLevel=3,
    criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 30, 0.01)
)
# ----------------------------------------------------------------------------

def dense_lucas_kanade(prev_g: np.ndarray, next_g: np.ndarray) -> np.ndarray:
    """Berechne ein pseudodichtes Lucasâ€‘Kanadeâ€‘Flussfeld via Gittersampling."""
    h, w = prev_g.shape
    ys, xs = np.mgrid[0:h:GRID_STEP, 0:w:GRID_STEP]
    p0 = np.stack((xs.flatten(), ys.flatten()), axis=-1).astype(np.float32)
    p0 = p0.reshape(-1, 1, 2)

    p1, st, _ = cv2.calcOpticalFlowPyrLK(prev_g, next_g, p0, None, **LK_ARGS)
    st = st.reshape(-1)
    p0 = p0.reshape(-1, 2)
    p1 = p1.reshape(-1, 2)

    flow = np.zeros((h, w, 2), dtype=np.float32)
    flow_valid = p1[st == 1] - p0[st == 1]
    pts_valid = p0[st == 1].astype(int)

    flow[pts_valid[:, 1], pts_valid[:, 0]] = flow_valid
    # GauÃŸâ€‘Blur glÃ¤ttet & fÃ¼llt LÃ¼cken â†’ pseudodicht
    flow[..., 0] = cv2.GaussianBlur(flow[..., 0], (SMOOTH_K, SMOOTH_K), 0)
    flow[..., 1] = cv2.GaussianBlur(flow[..., 1], (SMOOTH_K, SMOOTH_K), 0)
    return flow


def flow_speed(flow: np.ndarray) -> np.ndarray:
    dx, dy = flow[..., 0], flow[..., 1]
    return np.sqrt(dx ** 2 + dy ** 2)


def render_defmap(speed: np.ndarray, frame_no: int) -> np.ndarray:
    """Render Matplotlibâ€‘Figure â†’ RGBâ€‘Array."""
    fig = plt.figure(figsize=(4, 4), dpi=FIG_DPI)
    ax = fig.add_axes([0.05, 0.05, 0.8, 0.9])
    im = ax.imshow(speed, cmap="plasma", vmin=0, vmax=MAX_SPEED_VIS)
    ax.set_axis_off()
    ax.set_title(f"DefMap at Time: {frame_no}", fontsize=14, weight="bold")

    cb_ax = fig.add_axes([0.88, 0.1, 0.03, 0.8])
    cbar = fig.colorbar(im, cax=cb_ax)
    cbar.set_ticks([0, MAX_SPEED_VIS])
    cbar.set_ticklabels(["0", f"{int(MAX_SPEED_VIS)}"])

    fig.canvas.draw()
    w_canvas, h_canvas = fig.canvas.get_width_height()
    rgb = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
    plt.close(fig)
    return rgb.reshape((h_canvas, w_canvas, 3))


# ----------------------------------------------------------------------------
# MAIN PIPELINE
# ----------------------------------------------------------------------------
cap = cv2.VideoCapture(str(INPUT_VIDEO))
if not cap.isOpened():
    raise IOError(f"Cannot open {INPUT_VIDEO}")

ret, frame = cap.read()
if not ret:
    raise RuntimeError("Input video empty")

prev_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)[ROI_MARGIN:-ROI_MARGIN, ROI_MARGIN:-ROI_MARGIN]

# initial dummyâ€‘figure â†’ dimension for VideoWriter
frame0_rgb = render_defmap(np.zeros_like(prev_gray, dtype=float), 0)
H_fig, W_fig, _ = frame0_rgb.shape
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
writer = cv2.VideoWriter(str(VIDEO_OUT), fourcc, FPS_OUT, (W_fig, H_fig))

metrics = []
frame_idx = 0

while True:
    # evtl. Frames Ã¼berspringen
    for _ in range(DOWNSAMPLE):
        ret, frame = cap.read()
        if not ret:
            break
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)[ROI_MARGIN:-ROI_MARGIN, ROI_MARGIN:-ROI_MARGIN]
    flow = dense_lucas_kanade(prev_gray, gray)
    speed = flow_speed(flow)

    rgb_fig = render_defmap(speed, frame_idx)
    writer.write(cv2.cvtColor(rgb_fig, cv2.COLOR_RGB2BGR))

    metrics.append({
        "frame": frame_idx,
        "speed_mean": float(np.mean(speed)),
        "speed_std": float(np.std(speed)),
        "speed_max": float(np.max(speed)),
    })

    prev_gray = gray
    frame_idx += 1

cap.release()
writer.release()

pd.DataFrame(metrics).to_csv(CSV_OUT, index=False)
print(f"âœ… DefMap (Lucasâ€‘Kanade) video â†’ {VIDEO_OUT}\nðŸ“ˆ Metrics CSV â†’ {CSV_OUT}")


/var/folders/fy/j2skn70j52dfcbvjgdk6phr00000gn/T/ipykernel_69798/447498996.py:106: MatplotlibDeprecationWarning: The tostring_rgb function was deprecated in Matplotlib 3.8 and will be removed in 3.10. Use buffer_rgba instead.
  rgb = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
/var/folders/fy/j2skn70j52dfcbvjgdk6phr00000gn/T/ipykernel_69798/447498996.py:106: MatplotlibDeprecationWarning: The tostring_rgb function was deprecated in Matplotlib 3.8 and will be removed in 3.10. Use buffer_rgba instead.
  rgb = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
/var/folders/fy/j2skn70j52dfcbvjgdk6phr00000gn/T/ipykernel_69798/447498996.py:106: MatplotlibDeprecationWarning: The tostring_rgb function was deprecated in Matplotlib 3.8 and will be removed in 3.10. Use buffer_rgba instead.
  rgb = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
/var/folders/fy/j2skn70j52dfcbvjgdk6phr00000gn/T/ipykernel_69798/447498996.py:106: MatplotlibDeprecationWarning: The tostring_r

âœ… DefMap (Lucasâ€‘Kanade) video â†’ /Users/macbook/Library/Mobile Documents/com~apple~CloudDocs/Studium/Spezialisierung/CodeExampleRayan/data/output/defmap_video_lk/defmap_lk_plasma.mp4
ðŸ“ˆ Metrics CSV â†’ /Users/macbook/Library/Mobile Documents/com~apple~CloudDocs/Studium/Spezialisierung/CodeExampleRayan/data/output/defmap_video_lk/defmap_lk_speed_metrics.csv


/var/folders/fy/j2skn70j52dfcbvjgdk6phr00000gn/T/ipykernel_69798/447498996.py:106: MatplotlibDeprecationWarning: The tostring_rgb function was deprecated in Matplotlib 3.8 and will be removed in 3.10. Use buffer_rgba instead.
  rgb = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
/var/folders/fy/j2skn70j52dfcbvjgdk6phr00000gn/T/ipykernel_69798/447498996.py:106: MatplotlibDeprecationWarning: The tostring_rgb function was deprecated in Matplotlib 3.8 and will be removed in 3.10. Use buffer_rgba instead.
  rgb = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
